In [6]:
import mlflow
from mlflow.tracking import MlflowClient
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
import os
import requests
import json
import numpy as np
import time

# Funciones para traer base de datos

In [7]:
requests.get('http://host.docker.internal/restart_data_generation?group_number=2')
i = 0
columnas = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type', 'Cover_Type']

# Crear DataFrame vacío
df = pd.DataFrame(columns=columnas)

while i <10:
    try:
        r = requests.get('http://host.docker.internal:80/data?group_number=2')
        d = json.loads(r.content.decode('utf-8'))
        if i == d['batch_number']:
            print("Batch repetido. No se añaden datos")
        else:
            i = d['batch_number']
            print(f"Añadiendo datos batch {i}")
            temp = pd.DataFrame(d['data'], columns=columnas)
            # Concatenate all at once
            df = pd.concat([df, temp], ignore_index=True)


    except requests.exceptions.ConnectionError:
        print("🚫 No se pudo conectar a la API, reintentando...")
        
    if i==10:
        print("Todos los datos han sido guardados")
    else:
        time.sleep(3)  

Añadiendo datos batch 1
Batch repetido. No se añaden datos
Añadiendo datos batch 2
Batch repetido. No se añaden datos
Añadiendo datos batch 3
Batch repetido. No se añaden datos
Añadiendo datos batch 4
Batch repetido. No se añaden datos
Añadiendo datos batch 5
Batch repetido. No se añaden datos
Añadiendo datos batch 6
Batch repetido. No se añaden datos
Añadiendo datos batch 7
Batch repetido. No se añaden datos
Añadiendo datos batch 8
Batch repetido. No se añaden datos
Añadiendo datos batch 9
Batch repetido. No se añaden datos
Añadiendo datos batch 10
Todos los datos han sido guardados


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58100 entries, 0 to 58099
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Elevation                           58100 non-null  object
 1   Aspect                              58100 non-null  object
 2   Slope                               58100 non-null  object
 3   Horizontal_Distance_To_Hydrology    58100 non-null  object
 4   Vertical_Distance_To_Hydrology      58100 non-null  object
 5   Horizontal_Distance_To_Roadways     58100 non-null  object
 6   Hillshade_9am                       58100 non-null  object
 7   Hillshade_Noon                      58100 non-null  object
 8   Hillshade_3pm                       58100 non-null  object
 9   Horizontal_Distance_To_Fire_Points  58100 non-null  object
 10  Wilderness_Area                     58100 non-null  object
 11  Soil_Type                           58100 non-null  ob

In [10]:
df['Cover_Type'].value_counts()

Cover_Type
1    41409
0    16327
4      364
Name: count, dtype: int64

# Funciones para preprocesar datos

In [45]:
# Selección de variables predictoras y objetivo
feature_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
target_col = "Cover_Type"

X = df[feature_cols]
y = df[target_col]

# Dividir en entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Registrar modelos en MLFlow

In [37]:
client = MlflowClient()
for exp in client.search_experiments(view_type="ALL"):
    print(exp.experiment_id, exp.name, exp.lifecycle_stage)

3 Gradient_Boosting_looking_for_the_best_model deleted
2 Random_forest_looking_for_the_best_model deleted
1 mlflow_tracking_tipo_de_cubierta_forestal_proofs deleted
0 Default active


In [39]:
#client.restore_experiment(experiment_id="1")  # reemplaza con el ID real


In [40]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")
mlflow.set_experiment("mlflow_tracking_tipo_de_cubierta_forestal_proofs")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "the simplest possible example"

# executes the run
with mlflow.start_run(run_name="Random_forest_no_params", description=desc) as run:
    # Entrenar RandomForest
    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, y_train)
    
#mlflow.end_run()

2025/03/27 06:05:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/27 06:05:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [41]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")
mlflow.set_experiment("mlflow_tracking_tipo_de_cubierta_forestal_proofs")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "the simplest possible example"

# executes the run
with mlflow.start_run(run_name="Gradient_boost_no_params", description=desc) as run:
    # Entrenar GradietBoost
    gb_model = GradientBoostingClassifier()
    gb_model.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/27 06:05:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/27 06:06:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [42]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")
mlflow.set_experiment("Random_forest_looking_for_the_best_model")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "Random_forest_grid_search"

params = {
  "n_estimators": [33, 66, 200],
  "max_depth": [2, 4, 6],
  "max_features": [3, 4, 5]
}

rf_model = RandomForestClassifier()
searcher = GridSearchCV(estimator=rf_model, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)

with mlflow.start_run(run_name="autolog_with_grid_search") as run:
    searcher.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/27 06:06:14 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
/ml_project/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2025/03/27 06:07:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/27 06:07:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/27 06:07:41 INFO mlflow.sklearn.utils: Logging the 5 best runs, 22 runs will be omitted.


In [43]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")
mlflow.set_experiment("Gradient_Boosting_looking_for_the_best_model")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "Gradient_Boosting_grid_search"

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0]
}

gb_model = GradientBoostingClassifier()
searcher = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

with mlflow.start_run(run_name="autolog_with_grid_search") as run:
    searcher.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/27 06:07:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/27 06:23:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/27 06:23:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/27 06:23:51 INFO mlflow.sklearn.utils: Logging the 5 best runs, 31 runs will be omitted.


# Probar inferencia

**Probar inferencia mejor modelo de random forest**

In [58]:
label=2

In [59]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")

model_name = "Best_random_forest"

# logged_model = 'runs:/71428bebed2b4feb9635714ea3cdb562/model'
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=model_production_uri)
loaded_model
example_test = X_test.iloc[label].to_frame().T
#print(example_test)
print('real: ', y_test.iloc[label])
print('prediction: ', loaded_model.predict(example_test)[0])

real:  1
prediction:  1


**Probar inferencia mejor modelo de gradient boosting**


In [60]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://mlflow-service:5000")

model_name = "Best_gradient_boosting"

# logged_model = 'runs:/71428bebed2b4feb9635714ea3cdb562/model'
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=model_production_uri)
loaded_model
example_test = X_test.iloc[label].to_frame().T
#print(example_test)
print('real: ', y_test.iloc[label])
print('prediction: ', loaded_model.predict(example_test)[0])

real:  1
prediction:  1


In [61]:
X_test.iloc[0].to_frame()

,23884
Elevation,3172
Aspect,78
Slope,21
Horizontal_Distance_To_Hydrology,513
Vertical_Distance_To_Hydrology,97
Horizontal_Distance_To_Roadways,4350
Hillshade_9am,242
Hillshade_Noon,195
Hillshade_3pm,74
Horizontal_Distance_To_Fire_Points,3271


In [62]:
client = MlflowClient()

# List all registered models using search_registered_models
registered_models = client.search_registered_models()

# Print the names of the models
for model in registered_models:
    print(model.name)

Best_gradient_boosting
Best_random_forest


In [63]:
print(registered_models)

[<RegisteredModel: aliases={}, creation_timestamp=1743057193182, description='', last_updated_timestamp=1743057208117, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1743057193269, current_stage='Production', description='', last_updated_timestamp=1743057208117, name='Best_gradient_boosting', run_id='46d4643a47434854b58efdc08bd112bf', run_link='', source='s3://mlflows3/artifacts/3/46d4643a47434854b58efdc08bd112bf/artifacts/best_estimator', status='READY', status_message='', tags={}, user_id='', version='1'>], name='Best_gradient_boosting', tags={}>, <RegisteredModel: aliases={}, creation_timestamp=1743057159771, description='', last_updated_timestamp=1743057219347, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1743057159820, current_stage='Production', description='', last_updated_timestamp=1743057219347, name='Best_random_forest', run_id='33c6cb53c730493eb91eb149cbff5b8a', run_link='', source='s3://mlflows3/artifacts/2/33c6cb53c730493eb91eb149cbff5b8a/